### This notebook acts as an prototype in building the model Vocab

In [80]:
with open('input.txt', 'r') as f:
    text = f.read()

In [81]:
print("len of the dataset in a characters: ", len(text))

len of the dataset in a characters:  1115393


In [82]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [83]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Converting Raw text to sequence of Intergers

In [84]:
# create a mapping of characters to integers
stoi = { ch:i for i, ch in enumerate(chars) } # string to integer
itos = { i:ch for i, ch in enumerate(chars) } # integer to string

encode = lambda s: [stoi[c] for c in s] # encoder : take a string, output a list of integer
decode = lambda l: ''.join([itos[i] for i in l]) # decoder : take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


### Tokenization, train/val split

In [85]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
# train and test splits:

n = int(0.9 * len(data)) # first 90% will be train, rest 10% will be test
train_data = data[:n]
test_data = data[n:]

: 

In [86]:
block_size = 8
train_data[:block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [87]:
# this is a code to illustrate the concept of block size in a language model

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target  = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [88]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process parallel???
block_size = 8 # what is the maximum context length for predictions ???

def get_batch(split):
    # generate a small batch of data of inputs x and y targets:
    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    
    return x, y

xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

torch.Size([4, 8])
tensor([[56, 41, 46,  1, 53,  5,  1, 58],
        [14, 33, 15, 23, 21, 26, 19, 20],
        [58, 46, 43, 56, 10,  0, 14, 59],
        [47, 60, 43,  1, 51, 43,  1, 58]])
torch.Size([4, 8])
tensor([[41, 46,  1, 53,  5,  1, 58, 46],
        [33, 15, 23, 21, 26, 19, 20, 13],
        [46, 43, 56, 10,  0, 14, 59, 58],
        [60, 43,  1, 51, 43,  1, 58, 46]])


In [89]:
for b in range(batch_size): # batch dims
    for t in range(block_size): # time dims
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"when i/p is {context.tolist()} o/p is {target}")

when i/p is [56] o/p is 41
when i/p is [56, 41] o/p is 46
when i/p is [56, 41, 46] o/p is 1
when i/p is [56, 41, 46, 1] o/p is 53
when i/p is [56, 41, 46, 1, 53] o/p is 5
when i/p is [56, 41, 46, 1, 53, 5] o/p is 1
when i/p is [56, 41, 46, 1, 53, 5, 1] o/p is 58
when i/p is [56, 41, 46, 1, 53, 5, 1, 58] o/p is 46
when i/p is [14] o/p is 33
when i/p is [14, 33] o/p is 15
when i/p is [14, 33, 15] o/p is 23
when i/p is [14, 33, 15, 23] o/p is 21
when i/p is [14, 33, 15, 23, 21] o/p is 26
when i/p is [14, 33, 15, 23, 21, 26] o/p is 19
when i/p is [14, 33, 15, 23, 21, 26, 19] o/p is 20
when i/p is [14, 33, 15, 23, 21, 26, 19, 20] o/p is 13
when i/p is [58] o/p is 46
when i/p is [58, 46] o/p is 43
when i/p is [58, 46, 43] o/p is 56
when i/p is [58, 46, 43, 56] o/p is 10
when i/p is [58, 46, 43, 56, 10] o/p is 0
when i/p is [58, 46, 43, 56, 10, 0] o/p is 14
when i/p is [58, 46, 43, 56, 10, 0, 14] o/p is 59
when i/p is [58, 46, 43, 56, 10, 0, 14, 59] o/p is 58
when i/p is [47] o/p is 60
when i

In [90]:
print(xb)

tensor([[56, 41, 46,  1, 53,  5,  1, 58],
        [14, 33, 15, 23, 21, 26, 19, 20],
        [58, 46, 43, 56, 10,  0, 14, 59],
        [47, 60, 43,  1, 51, 43,  1, 58]])


### Simple Bigram Model via Pytorch

In [91]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [92]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits of the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None): 
        # idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx)  # (B, T, C)
        B, T, C = logits.shape

        if targets is None:
            loss = None
        else:
            # flatten the logits and targets for cross-entropy
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context:
        for _ in range(max_new_tokens):
            # get the predictions:    
            logits, loss = self(idx)
            # focus only on the last time step:
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probablities
            probs = F.softmax(logits, dim=1)
            # sample from the distributions:
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequneces
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T + 1)
        return idx 

In [93]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5907, grad_fn=<NllLossBackward0>)


In [94]:

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


### Optimizer

In [110]:
class SimpleAdam:
    def __init__(self, params, lr=1e-3, weight_decay=0):
        self.params = list(params)
        self.lr = lr
        self.weight_decay = weight_decay
        self.step_count = 0
        self.m = [torch.zeros_like(p) for p in self.params]
        self.v = [torch.zeros_like(p) for p in self.params]
        
    def zero_grad(self, set_to_none=False):
        for p in self.params:
            if p.grad is not None:
                if set_to_none:
                    p.grad = None
                else:
                    p.grad.zero_()
    
    def step(self):
        self.step_count += 1
        for i, p in enumerate(self.params):
            if p.grad is None:
                continue
            
            grad = p.grad.data
            if self.weight_decay != 0:
                grad = grad.add(p.data, alpha=self.weight_decay)
            
            # Adam update
            self.m[i] = 0.9 * self.m[i] + 0.1 * grad
            self.v[i] = 0.999 * self.v[i] + 0.001 * grad**2
            
            m_hat = self.m[i] / (1 - 0.9**self.step_count)
            v_hat = self.v[i] / (1 - 0.999**self.step_count)
            
            p.data.add_(m_hat / (torch.sqrt(v_hat) + 1e-8), alpha=-self.lr)

# Create optimizer
optimizer = SimpleAdam(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data:
    xb , yb = get_batch('train')
    
    # evaluate the loss 
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    print(loss.item())

In [119]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



LI d y, deare ar lesiseavimyowhr olrd
KIES:
S:
DWhe te ousee:
BERI ts ad dsod t l youse MIXYo the ouro s.

Wheape baris alllear wer n wangu irele derenthyor, mo s bu ik:
Thit; trs ay vel,
Coff morexfacang sofaror
LUSinsest o cit the isor, chatheyo yowive ys; hy we shy m hiremyo my.
Mas, forimy,
win
